In [1]:
# import numpy as np
# import os
# from PIL import Image
# import matplotlib.pyplot as plt

# Import functions

In [2]:
from wfa.ml_logic.model import load_data, shuffle_data, data_split, plot_loss_accuracy
from wfa.ml_logic.registry import save_model, load_model

2022-12-13 09:07:24.993526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 09:07:25.271310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-13 09:07:25.271424: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-13 09:07:26.598959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

## parameters of each function
- load_data(data_path:str)
- shuffle_data(X,y, seed=None)
- data_split(X,y,val_perc, test_perc)
- plot_loss_accuracy(history, title=None)
- save_model(model: Model = None, params: dict = {}, metrics: dict = {})
- load_model(save_copy_locally=False)

# Import / shuffle / split data

In [3]:
data_path = '../raw_data/EuroSAT'
X, y = load_data(data_path)
X, y = shuffle_data(X, y)
X_train, X_val, X_test, y_train, y_val, y_test = data_split(X,y,0.1,0.2)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:05<00:00, 514.59it/s]


# Create Model

In [4]:
# #import librairies
# import tensorflow as tf
# from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# from tensorflow.keras import models

## Model Parameters

In [5]:
epochs = 1
patience = 5
learning_rate = 1e-4
batch_size=32
n_conv_2D = 2
conv_2D_1 = 16
conv_2D_2 = 32
conv_2D_3 = 64
ks_1 = (3,3)
ks_2 = (2,2)
ks_3 = (2,2)
Dense_1 = 50

params = dict(epochs = epochs, patience=patience, learning_rate=learning_rate, batch_size=batch_size,
              n_conv_2D=n_conv_2D, conv_2D_1=conv_2D_1, conv_2D_2=conv_2D_2, conv_2D_3=conv_2D_3,
              ks_1=ks_1, ks_2=ks_2, ks_3=ks_3, 
              Dense_1=Dense_1
             )
params

{'epochs': 1,
 'patience': 5,
 'learning_rate': 0.0001,
 'batch_size': 32,
 'n_conv_2D': 2,
 'conv_2D_1': 16,
 'conv_2D_2': 32,
 'conv_2D_3': 64,
 'ks_1': (3, 3),
 'ks_2': (2, 2),
 'ks_3': (2, 2),
 'Dense_1': 50}

In [6]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(64,64,3)))

model.add(layers.Conv2D(conv_2D_1, kernel_size=ks_1, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=ks_1))

model.add(layers.Conv2D(conv_2D_2, kernel_size=ks_2, padding='same', activation="relu"))
model.add(layers.MaxPooling2D(pool_size=ks_2))

#model.add(layers.Conv2D(conv_2D_3, kernel_size=ks_3, padding='same', activation="relu"))
#model.add(layers.MaxPooling2D(pool_size=ks_3))

model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

opt = optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

2022-12-13 09:08:12.685431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-13 09:08:12.685484: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-13 09:08:12.685525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schneider-1091-luis-roberto-sant-anna-henriques): /proc/driver/nvidia/version does not exist
2022-12-13 09:08:12.685886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Fit Model

In [7]:
es = EarlyStopping(monitor = 'val_accuracy',
                patience = patience,
                restore_best_weights = True,
                verbose = 1,
                )

history = model.fit(X_train, y_train,
                            validation_data = (X_val, y_val),
                            batch_size = batch_size,
                            epochs=epochs,
                            callbacks=[es])


591/591 [==============================] - 57s 94ms/step - loss: 1.9490 - accuracy: 0.2734 - val_loss: 1.6216 - val_accuracy: 0.3970


## Get metrics and save model

In [8]:
val_accuracy = history.history['val_accuracy'][0]
test_accuracy = model.evaluate(X_test, y_test)[1]
metrics = dict(accuracy_val = val_accuracy, accuracy_test = test_accuracy)

169/169 [==============================] - 5s 28ms/step - loss: 1.6467 - accuracy: 0.4019


In [9]:
save_model(model, params=params, metrics=metrics)

INFO:tensorflow:Assets written to: /tmp/tmpvz7kt7h0/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpvz7kt7h0/model/data/model/assets
/home/luis.robertosantannahenriques/.pyenv/versions/3.10.6/envs/watching-from-above/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'wfa' already exists. Creating a new version of this model...
2022/12/13 09:09:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: wfa, version 5
Created version '5' of model 'wfa'.
